In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
application_df = application_df.drop(columns=["EIN", "NAME"])
application_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   APPLICATION_TYPE        34299 non-null  object
 1   AFFILIATION             34299 non-null  object
 2   CLASSIFICATION          34299 non-null  object
 3   USE_CASE                34299 non-null  object
 4   ORGANIZATION            34299 non-null  object
 5   STATUS                  34299 non-null  int64 
 6   INCOME_AMT              34299 non-null  object
 7   SPECIAL_CONSIDERATIONS  34299 non-null  object
 8   ASK_AMT                 34299 non-null  int64 
 9   IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 2.6+ MB


In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_type_counts = application_df["APPLICATION_TYPE"].value_counts()
print(application_type_counts)

# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_type_counts[application_type_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [5]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df["CLASSIFICATION"].value_counts()
print(classification_counts)

# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [6]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,108590,1,False,False,False,True,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,1,5000,0,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
3,1,6692,1,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,1,142590,1,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,True,False


In [7]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"]

# Define feature variables
X = application_df.drop(columns=["IS_SUCCESSFUL"])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

# Display shapes of training and testing sets
X_train.shape, X_test.shape

((25724, 43), (8575, 43))

In [8]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = X_train.shape[1]

# Create a new optimized model
optimized_nn = tf.keras.models.Sequential()

# First hidden layer (more neurons)
optimized_nn.add(tf.keras.layers.Dense(units=100, activation="relu", input_dim=input_features))

# Second hidden layer (changed activation function)
optimized_nn.add(tf.keras.layers.Dense(units=50, activation="tanh"))

# Third hidden layer (extra layer)
optimized_nn.add(tf.keras.layers.Dense(units=25, activation="relu"))

# Output layer
optimized_nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the optimized model
optimized_nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the optimized model
optimized_history = optimized_nn.fit(X_train_scaled, y_train, epochs=75, validation_data=(X_test_scaled, y_test))

# Evaluate the optimized model
opt_loss, opt_accuracy = optimized_nn.evaluate(X_test_scaled, y_test)

print(f"Optimized Model Loss: {opt_loss:.4f}")
print(f"Optimized Model Accuracy: {opt_accuracy:.4f}")

Epoch 1/75
804/804 [==============================] - 1s 705us/step - loss: 0.5641 - accuracy: 0.7255 - val_loss: 0.5587 - val_accuracy: 0.7251
Epoch 2/75
804/804 [==============================] - 1s 691us/step - loss: 0.5542 - accuracy: 0.7285 - val_loss: 0.5599 - val_accuracy: 0.7255
Epoch 3/75
804/804 [==============================] - 1s 660us/step - loss: 0.5517 - accuracy: 0.7306 - val_loss: 0.5619 - val_accuracy: 0.7198
Epoch 4/75
804/804 [==============================] - 1s 655us/step - loss: 0.5496 - accuracy: 0.7310 - val_loss: 0.5583 - val_accuracy: 0.7251
Epoch 5/75
804/804 [==============================] - 1s 637us/step - loss: 0.5489 - accuracy: 0.7325 - val_loss: 0.5579 - val_accuracy: 0.7266
Epoch 6/75
804/804 [==============================] - 1s 653us/step - loss: 0.5482 - accuracy: 0.7321 - val_loss: 0.5552 - val_accuracy: 0.7283
Epoch 7/75
804/804 [==============================] - 1s 721us/step - loss: 0.5471 - accuracy: 0.7343 - val_loss: 0.5555 - val_accuracy:

In [11]:
# Create a new optimized model
improved_nn = tf.keras.models.Sequential()

# First hidden layer (more neurons & activation function change)
improved_nn.add(tf.keras.layers.Dense(units=128, activation="relu", input_dim=input_features))
improved_nn.add(tf.keras.layers.Dropout(0.2))  # Add dropout

# Second hidden layer (increased neurons & Leaky ReLU)
improved_nn.add(tf.keras.layers.Dense(units=64, activation=tf.keras.layers.LeakyReLU(alpha=0.01)))
improved_nn.add(tf.keras.layers.Dropout(0.2))

# Third hidden layer (extra layer)
improved_nn.add(tf.keras.layers.Dense(units=32, activation="relu"))

# Output layer
improved_nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the improved model with a lower learning rate
improved_nn.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), metrics=["accuracy"])

# Train the improved model for 100 epochs
improved_history = improved_nn.fit(X_train_scaled, y_train, epochs=100, validation_data=(X_test_scaled, y_test))

# Evaluate improved model
imp_loss, imp_accuracy = improved_nn.evaluate(X_test_scaled, y_test)

print(f"Improved Model Loss: {imp_loss:.4f}")
print(f"Improved Model Accuracy: {imp_accuracy:.4f}")

improved_nn.save("AlphabetSoupCharity_Optimization.h5")

Epoch 1/100
804/804 [==============================] - 1s 744us/step - loss: 0.5863 - accuracy: 0.7087 - val_loss: 0.5637 - val_accuracy: 0.7269
Epoch 2/100
804/804 [==============================] - 1s 687us/step - loss: 0.5649 - accuracy: 0.7280 - val_loss: 0.5582 - val_accuracy: 0.7266
Epoch 3/100
804/804 [==============================] - 1s 720us/step - loss: 0.5600 - accuracy: 0.7296 - val_loss: 0.5591 - val_accuracy: 0.7280
Epoch 4/100
804/804 [==============================] - 1s 686us/step - loss: 0.5573 - accuracy: 0.7291 - val_loss: 0.5551 - val_accuracy: 0.7290
Epoch 5/100
804/804 [==============================] - 1s 672us/step - loss: 0.5565 - accuracy: 0.7293 - val_loss: 0.5540 - val_accuracy: 0.7266
Epoch 6/100
804/804 [==============================] - 1s 675us/step - loss: 0.5544 - accuracy: 0.7301 - val_loss: 0.5546 - val_accuracy: 0.7269
Epoch 7/100
804/804 [==============================] - 1s 670us/step - loss: 0.5540 - accuracy: 0.7304 - val_loss: 0.5544 - val_ac

/opt/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=200, random_state=1)
rf_model.fit(X_train_scaled, y_train)

# Predict on test data
y_pred = rf_model.predict(X_test_scaled)

# Evaluate accuracy
rf_accuracy = accuracy_score(y_test, y_pred)
print(f"Random Forest Model Accuracy: {rf_accuracy:.4f}")

Random Forest Model Accuracy: 0.7116


In [12]:
from sklearn.feature_selection import RFE

model = RandomForestClassifier(n_estimators=100, random_state=1)
selector = RFE(model, n_features_to_select=10)  # Keep only 10 most important features
selector.fit(X_train_scaled, y_train)

selected_features = X.columns[selector.support_]
print("Selected Features:", selected_features)

Selected Features: Index(['ASK_AMT', 'APPLICATION_TYPE_T10', 'APPLICATION_TYPE_T19',
       'APPLICATION_TYPE_T3', 'APPLICATION_TYPE_T4', 'APPLICATION_TYPE_T5',
       'AFFILIATION_CompanySponsored', 'AFFILIATION_Independent',
       'CLASSIFICATION_C2000', 'ORGANIZATION_Association'],
      dtype='object')


In [13]:
# Keeping only the selected features
X_selected = X[selected_features]

# Split into training and testing sets
X_train_selected, X_test_selected, y_train_selected, y_test_selected = train_test_split(X_selected, y, random_state=1, stratify=y)

# Scale the selected features
scaler_selected = StandardScaler()
X_train_selected_scaled = scaler_selected.fit_transform(X_train_selected)
X_test_selected_scaled = scaler_selected.transform(X_test_selected)

In [14]:
# Define a new model with selected features
selected_nn = tf.keras.models.Sequential()

# First hidden layer (fewer neurons since we have fewer features)
selected_nn.add(tf.keras.layers.Dense(units=32, activation="relu", input_dim=len(selected_features)))

# Second hidden layer
selected_nn.add(tf.keras.layers.Dense(units=16, activation="relu"))

# Output layer
selected_nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
selected_nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
selected_history = selected_nn.fit(X_train_selected_scaled, y_train_selected, epochs=75, validation_data=(X_test_selected_scaled, y_test_selected))

# Evaluate the model
selected_loss, selected_accuracy = selected_nn.evaluate(X_test_selected_scaled, y_test_selected)

print(f"Selected Features Model Loss: {selected_loss:.4f}")
print(f"Selected Features Model Accuracy: {selected_accuracy:.4f}")

Epoch 1/75
804/804 [==============================] - 1s 635us/step - loss: 0.5869 - accuracy: 0.7119 - val_loss: 0.5771 - val_accuracy: 0.7223
Epoch 2/75
804/804 [==============================] - 0s 551us/step - loss: 0.5740 - accuracy: 0.7213 - val_loss: 0.5736 - val_accuracy: 0.7215
Epoch 3/75
804/804 [==============================] - 0s 563us/step - loss: 0.5732 - accuracy: 0.7222 - val_loss: 0.5735 - val_accuracy: 0.7206
Epoch 4/75
804/804 [==============================] - 0s 559us/step - loss: 0.5725 - accuracy: 0.7222 - val_loss: 0.5739 - val_accuracy: 0.7244
Epoch 5/75
804/804 [==============================] - 0s 588us/step - loss: 0.5722 - accuracy: 0.7231 - val_loss: 0.5729 - val_accuracy: 0.7245
Epoch 6/75
804/804 [==============================] - 0s 552us/step - loss: 0.5717 - accuracy: 0.7227 - val_loss: 0.5714 - val_accuracy: 0.7243
Epoch 7/75
804/804 [==============================] - 0s 548us/step - loss: 0.5716 - accuracy: 0.7230 - val_loss: 0.5740 - val_accuracy:

In [15]:
# Train Random Forest on selected features
rf_selected = RandomForestClassifier(n_estimators=200, random_state=1)
rf_selected.fit(X_train_selected_scaled, y_train_selected)

# Predict on test data
rf_selected_pred = rf_selected.predict(X_test_selected_scaled)

# Evaluate accuracy
rf_selected_accuracy = accuracy_score(y_test_selected, rf_selected_pred)
print(f"Random Forest Model Accuracy (Selected Features): {rf_selected_accuracy:.4f}")

Random Forest Model Accuracy (Selected Features): 0.7037
